In [ ]:
import os

os.chdir("..")

In [ ]:
import geopandas as gpd
from src.inserts import data_inserts 
import polars as pl
import requests
import sqlite3
import duckdb
di = data_inserts()

In [ ]:
di.data

In [ ]:
# di.insert_regions()
# di.insert_states()
# di.insert_divisions()
# di.insert_county()
# di.insert_track()
# di.insert_datasets()
# di.insert_years()
di.insert_geo_full()

In [ ]:
df = di.conn.execute("""
   SELECT *
        FROM sqlite_db.dataset_table;
""").pl()
df.filter(pl.col("dataset").str.starts_with("basic"))

In [ ]:
df = di.data.with_columns(
    test = pl.col("c_geographyLink")
        .str.split("/")
        .list.get(4)
        .fill_null("")
)
df.filter(test="timeseries")

In [ ]:
df = di.conn.execute("""
   SELECT *
        FROM sqlite_db.geo_interm;
""").pl()
df

In [ ]:
1166.28/5

In [ ]:
1

In [ ]:
di.data.with_columns(test = pl.col("c_geographyLink").str.split('/').list.slice(4))

In [ ]:
di.data.select("c_geographyLink").to_series().to_list()

In [ ]:
base = "http://api.census.gov/data/2012/ecnlifomine/geography.json"
results = requests.get(base).json().get("fips")

# Drop the wildcard field
clean = [{k: v for k, v in rec.items() if k != "wildcard"} for rec in results]

df = pl.DataFrame(clean)

df

In [ ]:
requests.get(base).json().get("fips")

In [ ]:
pl.DataFrame(requests.get(base).json().get("fips"), strict=False)

In [ ]:
df

In [ ]:
df.with_columns(pl.col("geoLevelDisplay").fill_null("-1"))

In [ ]:
if "geoLevelId" not in df.columns and "geoLevelDisplay" not in df.columns:
    print("yeas")

In [ ]:
base = "http://api.census.gov/data/2010/acs/flows/geography.json"
# results = requests.get(base).json()
df = pl.DataFrame(requests.get(base).json().get("fips"))
df.columns

In [ ]:
if "geoLevelId" in df.columns:
    df = df.with_columns(geoLevelDisplay=pl.col("geoLevelId"))
df

In [ ]:
df.select(pl.col("name") + "-" + pl.col("geoLevelDisplay")).to_series().to_list()

In [ ]:
for obs in df.select(pl.col("name") + "-" + pl.col("geoLevelDisplay")).to_series().to_list():
    geo_desc, geo_lv = obs.split("-")
    if di.get_geo_id(geo_lv=geo_lv) == -1:
        di.conn.execute(f"""
            INSERT INTO sqlite_db.geo_table 
                (geo_name, geo_lv) VALUES ('{geo_desc}', '{geo_lv}');
        """)
        print(f"inserted {geo_desc} into db")

    geo_id = di.get_geo_id(geo_lv=geo_lv)
    year_id = di.get_year_id(base.split("/")[4])
    dataset_id = di.get_dataset_id(base.split("/")[5])

    if di.check_geo_interm_id(dataset_id=dataset_id, geo_id=geo_id, year_id=year_id):
        continue
    else:
        di.insert_geo_interm(dataset_id=dataset_id, geo_id=geo_id, year_id=year_id)
    

In [ ]:
base.split("/")

In [ ]:
dataset = base.split("/")[5:-1]
fc.join(dataset)

In [ ]:
geo_desc = "us"
geo_lv = "010"
geo_id = di.get_geo_id(geo_lv=geo_lv)

year = base.split("/")[4]
year_id = di.get_year_id(base.split("/")[4])

dataset = base.split("/")[5:-1]
dataset_id = di.get_dataset_id("-".join(base.split("/")[5:-1]))

# query = f"INSERT INTO sqlite_db.geo_interm (dataset_id, geo_id, year_id) VALUES ({dataset_id}, {geo_id}, {year_id});"
# query2 = f"SELECT * FROM sqlite_db.geo_interm;"

# di.conn.execute(query=query2).pl()
dataset_id

In [ ]:
di.conn.execute("SELECT * FROM sqlite_db.geo_table;").pl()

In [ ]:
df.select("geoLevelDisplay").to_series().to_list()